In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from sklearn.model_selection import train_test_split as tts
from sklearn.neighbors import KNeighborsClassifier 
import joblib


(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=8,
    validation_data=ds_test,
)

Epoch 1/8
469/469 [==============================] - 3s 3ms/step - loss: 0.3447 - sparse_categorical_accuracy: 0.9059 - val_loss: 0.1873 - val_sparse_categorical_accuracy: 0.9454
Epoch 2/8
469/469 [==============================] - 1s 2ms/step - loss: 0.1607 - sparse_categorical_accuracy: 0.9542 - val_loss: 0.1318 - val_sparse_categorical_accuracy: 0.9618
Epoch 3/8
469/469 [==============================] - 1s 2ms/step - loss: 0.1162 - sparse_categorical_accuracy: 0.9667 - val_loss: 0.1108 - val_sparse_categorical_accuracy: 0.9675
Epoch 4/8
469/469 [==============================] - 1s 2ms/step - loss: 0.0902 - sparse_categorical_accuracy: 0.9741 - val_loss: 0.0962 - val_sparse_categorical_accuracy: 0.9709
Epoch 5/8
469/469 [==============================] - 1s 2ms/step - loss: 0.0727 - sparse_categorical_accuracy: 0.9790 - val_loss: 0.0845 - val_sparse_categorical_accuracy: 0.9749
Epoch 6/8
469/469 [==============================] - 1s 2ms/step - loss: 0.0607 - sparse_categorical_accu

In [7]:
from scipy.ndimage.interpolation import shift, rotate

mnist =tf.keras.datasets.mnist

(x_train,y_train),(x_test,y_test)=mnist.load_data()

x_train=tf.keras.utils.normalize(x_train,axis=1)
x_test=tf.keras.utils.normalize(x_test,axis=1)


# Method to shift the image by given dimension
def shift_image(image, dx, dy):
    #image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image

# Creating Augmented Dataset
x_train_augmented = [image for image in x_train]
y_train_augmented = [image for image in y_train]

for dx, dy in ((2,0),(1,0), (-1,0),(-2,0),(0,2), (0,1), (0,-1),(0,-2)):
    for image, label in zip(x_train, y_train):
        x_train_augmented.append(shift_image(image, dx, dy))     
        y_train_augmented.append(label)

for angle in [10,5,-5,-10]:
    for image, label in zip(x_train,y_train):
        x_train_augmented.append(rotate(image,angle))
        y_train_augmented.append(label)



model=tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(128,activation="relu"))
model.add(tf.keras.layers.Dense(128,activation="relu"))
model.add(tf.keras.layers.Dense(10,activation="softmax"))

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

model.fit(x_train,y_train,epochs=8)



C:\Users\DELL\AppData\Local\Temp\ipykernel_6244\2673763067.py:1: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift, rotate
C:\Users\DELL\AppData\Local\Temp\ipykernel_6244\2673763067.py:1: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift, rotate


Epoch 1/8
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2656 - accuracy: 0.9213
Epoch 2/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1084 - accuracy: 0.9671
Epoch 3/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0749 - accuracy: 0.9765
Epoch 4/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0566 - accuracy: 0.9817
Epoch 5/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0421 - accuracy: 0.9863
Epoch 6/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0343 - accuracy: 0.9887
Epoch 7/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0258 - accuracy: 0.9912
Epoch 8/8
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0232 - accuracy: 0.9919


In [9]:
model.save("MNIST_NN.model")

INFO:tensorflow:Assets written to: MNIST_NN.model\assets


INFO:tensorflow:Assets written to: MNIST_NN.model\assets


In [8]:
loss,accuracy=model.evaluate(x_test,y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.1002 - accuracy: 0.9732
